In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
df['diagnosis'] = np.where(df.diagnosis == 'M',1,0)

In [ ]:
#df.head()

In [ ]:
df.diagnosis.value_counts()

In [ ]:
df.corr()['diagnosis'].sort_values(ascending=False)

In [ ]:
df = df.drop(columns=['id','Unnamed: 32'],axis=1)

In [ ]:
df.describe()

In [ ]:
df.corr()

# **DATA VISUALIZATION**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.express as px

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(df.corr(),annot=True,cmap='YlGnBu',center=0.4)

In [ ]:
columns = ['diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean', 'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean']

sns.pairplot(df[columns], hue="diagnosis", palette='mako')

In [ ]:
plt.figure(figsize=(10,8))
sns.countplot(x='diagnosis',data=df)

In [ ]:
fig = px.scatter_matrix(df, dimensions=['radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean'], color="diagnosis")
fig.show()

In [ ]:
fig = px.scatter(df, x='smoothness_mean', y='compactness_mean', color="diagnosis",
            log_x=True, size_max=60)
fig.show()

In [ ]:
fig = px.area(df, x="radius_worst", y="texture_worst", color="diagnosis")
fig.show()

In [ ]:
fig = px.histogram(df, x="perimeter_se", y="area_se", color="diagnosis", marginal="rug", hover_data=df.columns)
fig.show()

In [ ]:
fig = px.density_contour(df, x="concave points_worst", y="perimeter_worst",color="diagnosis", marginal_x="rug", marginal_y="histogram")
fig.show()

# **FEATURE SELECTION**

In [ ]:
df.columns

In [ ]:
X =df.drop('diagnosis',axis=1)


y = df.diagnosis

In [ ]:
## using pearson correlation 
plt.figure(figsize=(20,20))
cor = X.corr()
sns.heatmap(cor,annot=True,cmap = plt.cm.CMRmap_r)

In [ ]:
## with the following function we can select highly correlated features

def correlation(dataset,threshold):
    col_corr = set() # set of names of all the columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i,j]) > threshold:# we r intrested in coeff value
                col_name = corr_matrix.columns[i] # getting the name of column
                col_corr.add(col_name)
    return col_corr
                

In [ ]:
corr_features = correlation(X,0.9)
len(set(corr_features))

In [ ]:
corr_features

In [ ]:
X = X.drop(corr_features,axis=1)

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [ ]:
from sklearn.feature_selection import mutual_info_classif

from sklearn.feature_selection import SelectKBest

In [ ]:
mutual_info = mutual_info_classif(X_train,y_train)
mutual_info

In [ ]:
mutual_info = pd.Series(mutual_info)
mutual_info.index = X_train.columns
mutual_info.sort_values(ascending=False)

In [ ]:
mutual_info.sort_values(ascending=False).plot.bar(figsize=(20,13))

In [ ]:
select_15_best = SelectKBest(mutual_info_classif,k=15)
select_15_best.fit(X_train,y_train)
cols = X_train.columns[select_15_best.get_support()]


In [ ]:
X_train = X_train[cols] 
X_test = X_test[cols]

# **FEATURE SCALING**

In [ ]:
from sklearn.preprocessing import StandardScaler 
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)

 # **TRAINING AND PREDICTIONS** 

 #  1. USING STRATIFIED K_FOLD

In [ ]:
from sklearn.linear_model import SGDClassifier
sgd_cal = SGDClassifier()

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(sgd_cal,X_train,y_train,cv=10)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score

In [ ]:
score.mean()

In [ ]:
X_train.shape,y_train.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
accuracy = []
skfl = StratifiedKFold(n_splits=10,random_state=None)
skfl.get_n_splits(X_train,y_train)
for train_index,test_index in skfl.split(X_train,y_train):
    print('train:',train_index,'validation:',test_index)
    X1_train , X1_test = X_train[train_index] , X_train[test_index]
    y1_train , y1_test = y_train.iloc[train_index] , y_train.iloc[test_index]
    
    sgd_cal.fit(X1_train,y1_train)
    prediction = sgd_cal.predict(X1_test)
    score = accuracy_score(prediction,y1_test)
    accuracy.append(score)

In [ ]:
np.array(accuracy).mean()

# 2. USING LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(X_train,y_train)

In [ ]:
prediction = lr.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

In [ ]:
print(classification_report(y_test,prediction))

In [ ]:
sns.heatmap(confusion_matrix(y_test,prediction),annot=True)

In [ ]:
print(accuracy_score(y_test,prediction))

# 3. USING SGD CLASSIFIER

In [ ]:
from sklearn.linear_model import SGDClassifier

In [ ]:
sgd_cal = SGDClassifier()

In [ ]:
sgd_cal.fit(X_train,y_train)

In [ ]:
predictions = sgd_cal.predict(X_test)

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
sns.heatmap(confusion_matrix(y_test,predictions),annot=True,cmap='YlGnBu')

In [ ]:
print(accuracy_score(y_test,predictions))